In [3]:
import ollama



In [17]:
food_names = []

In [23]:
# Generate a list of food names


In [26]:
for i in range(10):
    # get pakistan food names
    stream = ollama.chat(
        model='llama3.1',
        messages=[{'role': 'user', 'content': 'generate a list of paksitani food. show just names no numbers or category headers. do not add any extra information just food names please.'}],
        stream=False,
    )

    # get result
    ans = stream['message']['content']

    # split in lines and record new food names
    for i in ans.split('\n'):
        if i not in food_names:
            food_names.append(i)
    print(len(food_names))

32
35
40
40
45
47
51
54
57
61


In [27]:
food_names

['Nihari',
 'Kebab',
 'Biryani',
 'Korma',
 'Naan',
 'Samosas',
 'Falooda',
 'Haleem',
 'Pakora',
 'Shawarma',
 'Tandoori Chicken',
 'Gulab Jamun',
 'Jalebi',
 'Sheermal',
 'Qeema',
 'Rogan Josh',
 'Makhni',
 'Kebabs',
 'Tikka',
 'Seekh',
 'Jalfrezi',
 'Nargisi Kofta',
 ' Haleem',
 'Karahi',
 'Kachoris',
 'Barfi',
 'Malai Kofta',
 'Shami Kabab',
 'Aloo Gobi',
 'Paratha',
 'Naan Bread',
 'Sag Aloo',
 'Seekh Kebab',
 'Qorma',
 'Shahi Tukray',
 'Karachi Breeze',
 'Shami Kebab',
 'Halwa Puri',
 'Chana Masala',
 'Nihari Kebab',
 'Karachi Haleem',
 'Farchi',
 'Kachoris ',
 'Sag Aloo ',
 'Zarda',
 'Karachi Bites',
 'Chicken Tikka',
 'Samosa',
 'Fulka',
 'Halwa',
 'Sheesh Mahal',
 'Sheesh Kabab',
 'Baingan Barta',
 'Kofta',
 'Chicken Karahi',
 'Baingan Bharta',
 'Kulfi',
 'Halwa Poori',
 'Karahi Chicken',
 'Pakora ',
 'Khichdi']

In [63]:
description = []
# generate descriptions 
for name in food_names:
    # get pakistan food names
    stream = ollama.chat(
        model='llama3.1',
        messages=[{'role': 'user', 'content': 'generate a  short description of the following pakistani food: '+name}],
        stream=False,
    )

    # get result
    ans = stream['message']['content']

    description.append(ans)

In [84]:
import numpy as np

embeddings = {}

e1 = np.array(ollama.embeddings(model='llama3.1', prompt='pakistani food')['embedding'])
for i in food_names:
    e2 = np.array(ollama.embeddings(model='llama3.1', prompt=i)['embedding'])
    embeddings[i] = e2

In [88]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    return dot_product / (magnitude_v1 * magnitude_v2)

In [91]:
for n in embeddings:
    e2 = embeddings[n]
    print(n , cosine_similarity(e1, e2))  # Calculate cosine similarity between e1 and e2


Nihari 0.5509102474103875
Kebab 0.5290328916628928
Biryani 0.5550861994186617
Korma 0.4908160967183946
Naan 0.4187024020102248
Samosas 0.5417133542504685
Falooda 0.5197471368711961
Haleem 0.41607539078803135
Pakora 0.5704251772625349
Shawarma 0.5095020060794804
Tandoori Chicken 0.5128664238583371
Gulab Jamun 0.48301782158848877
Jalebi 0.5085489837446261
Sheermal 0.5161759604445885
Qeema 0.435554426929487
Rogan Josh 0.5017786915847088
Makhni 0.4242991814327487
Kebabs 0.5506972917077915
Tikka 0.45858124054177013
Seekh 0.4332211596532384
Jalfrezi 0.5038248656778498
Nargisi Kofta 0.4175616731485238
 Haleem 0.4796161440617232
Karahi 0.5430720680322614
Kachoris 0.5211313678878913
Barfi 0.5128277672277802
Malai Kofta 0.44741449759234886
Shami Kabab 0.539609855812857
Aloo Gobi 0.3524918740040829
Paratha 0.5375652134554247
Naan Bread 0.4965790620627148
Sag Aloo 0.38090689377232956
Seekh Kebab 0.4902856278246479
Qorma 0.4681814427256407
Shahi Tukray 0.5063798460268345
Karachi Breeze 0.4557890397

In [30]:
# talk to stable diffusion
import webuiapi
# create API client
api = webuiapi.WebUIApi()

In [62]:
for food_name in food_names:
    result1 = api.txt2img(prompt=food_name,
                        negative_prompt="ugly, out of frame",
                        seed=1003,
                        styles=["anime"],
                        cfg_scale=7,

                        )
    name = food_name.lower().replace(' ', '_')
    # images contains the returned images (PIL images)
    result1.image.save(f'./public/food/{name}.webp')

In [92]:
data = []
# write to json
for idx, name in enumerate(food_names):
    desc = description[idx]
    desc = desc.replace("*", '').split('\n')[1:]
    desc[1] = desc[1][13:]
    desc = ''.join([i for i in desc])
    _name = name.lower().replace(' ', '_')
    img = '/food/'+_name +'.webp'
    
    data.append([idx, name, desc, img])

In [93]:
import json
# Convert data to a JSON string
json_data = json.dumps(data)

# Print the JSON string
print(json_data)

# Save the JSON data to a file called 'output.json'
with open('./src/data/menu.json', 'w') as f:
    json.dump(data, f)

[[0, "Nihari", "Nihari is a traditional, slow-cooked stew originating from Mughal India and popular in Pakistan. Made with tender chunks of meat (usually beef or lamb), this flavorful dish is cooked for several hours to achieve a rich, velvety texture. Aromatics like onions, garlic, ginger, and spices are saut\u00e9ed before being added to the pot along with bone marrow, which enriches the broth's flavor and body. The result is a hearty, comforting meal that warms the soul.", "/food/nihari.webp"], [1, "Kebab", "A popular Pakistani street food, Kebabs are skewers of marinated meat (usually lamb or beef) grilled to perfection. The tender, juicy meat is infused with a blend of spices, herbs, and sometimes yogurt, creating a flavorful and aromatic snack that's often served with a side of naan bread, chutneys, or salads.", "/food/kebab.webp"], [2, "Biryani", "assic Pakistani dish, Biryani is a flavorful and aromatic rice-based meal made with layers of basmati rice, marinated meat (usually l